In [1]:
import torch 
import cv2
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

from process import *
from drawbox import *
from framedif import *
from collections import deque
    
import os
import sys
sys.path.append('../va_pipeline/')

%matplotlib inline


In [2]:
model = torch.hub.load('ultralytics/yolov5', 'yolov5n')

Using cache found in C:\Users\adamf/.cache\torch\hub\ultralytics_yolov5_master
YOLOv5  2023-6-26 Python-3.9.13 torch-2.0.1+cpu CPU

Fusing layers... 
YOLOv5n summary: 213 layers, 1867405 parameters, 0 gradients
Adding AutoShape... 


In [3]:
def process_frame_diff_alternate(frame, frame_no, queue):
    if len(queue) != 3:
        queue.append(frame)
        return False, queue
        
    if frame_no % 3 == 0:
        dq = deque(queue)
        dq.popleft()
        dq.append(frame)
        return True, list(dq)
    else: 
        return False, queue

    

In [34]:
X_RES = 1920
Y_RES = 1080
cap = cv2.VideoCapture('../samples/medium.mp4')
out = cv2.VideoWriter(
    './test.mp4', 
    cv2.VideoWriter_fourcc(*'mp4v'), 
    30,
    (X_RES, Y_RES)
    )
thresh = 250

In [35]:
queue = deque([cap.read()[1], cap.read()[1], cap.read()[1]])
no_read = 1

prev = model(queue[0]).pandas().xywh[0]

while no_read <= thresh:
    ret, frame = cap.read()
    
    if not ret:  # no more frames to read
        print('no more frames')
        break
    else:
        no_read += 1

    # process, queue = process_frame_diff_alternate(frame, no_read, queue)
        
    # if process:
    #     diff = frame_diff(queue[0], queue[1], queue[2])
    #     output = cropped_detection(model, diff, queue[2])
    #     prev = output
    # else:
    #     output = prev
    
    queue.popleft()
    queue.append(frame)
    
    diff = frame_diff(queue[0], queue[1], queue[2])
    edge = get_frame_feature(diff)
    x,y,w,h = cv2.boundingRect(edge)
    
    if w < 100 or h < 100:
        output = prev
    else:
        cropped = queue[1][y:(y + h), x:(x + w)]
        output = model(cropped).pandas().xywh[0]
        prev = output
        output['xcenter'] += x
        output['ycenter'] += y
    
    for _, row in output.iterrows():
        x_center, y_center, width, height = row['xcenter'], row['ycenter'], row['width'], row['height']

        # Get top left corner coordinates
        topLeft = (int(x_center - width/2), int(y_center - height/2))
        bottomRight = (int(x_center + width/2), int(y_center + height/2))

        # Draw bounding box
        cv2.rectangle(diff, topLeft, bottomRight, (255, 0, 255), 2)

    out.write(diff)

cap.release()
out.release()

Ground truth

In [5]:
X_RES = 1920
Y_RES = 1080
cap = cv2.VideoCapture('../samples/medium.mp4')
out = cv2.VideoWriter(
    './test_ground.mp4', 
    cv2.VideoWriter_fourcc(*'mp4v'), 
    30,
    (X_RES, Y_RES)
    )

model = torch.hub.load('ultralytics/yolov5', 'yolov5n')

while cap.isOpened():
    ret, frame = cap.read()

    if not ret:  # no more frames to read
        break
    
    output = model(frame).pandas().xywh[0]
    
    for _, row in output.iterrows():
        x_center, y_center, width, height = row['xcenter'], row['ycenter'], row['width'], row['height']

        # Get top left corner coordinates
        topLeft = (int(x_center - width/2), int(y_center - height/2))
        bottomRight = (int(x_center + width/2), int(y_center + height/2))

        # Draw bounding box
        cv2.rectangle(frame, topLeft, bottomRight, (255, 0, 255), 2)

    out.write(frame)
    
    
cap.release()
out.release()

Using cache found in C:\Users\adamf/.cache\torch\hub\ultralytics_yolov5_master
YOLOv5  2023-6-26 Python-3.9.13 torch-2.0.1+cpu CPU

Fusing layers... 
YOLOv5n summary: 213 layers, 1867405 parameters, 0 gradients
Adding AutoShape... 
